In [34]:
### generate complex data 
import numpy as np
np.random.seed(5)

def generate_data(n_samples, true_coefficients, noise_level=0.1, poly_power=0.05):

    # Generate a random input signal
    k = len(true_coefficients[0])
    m, n = true_coefficients[0][0].shape
    p = len(true_coefficients[1])
    con = true_coefficients[0]
    w = true_coefficients[1]


    x = np.random.randn(n_samples, m) + 1j*np.random.randn(n_samples, m)

    # middle layer : convolution
    u = np.zeros((n_samples, n), dtype=np.complex128)
    for j in range(k):
        u += np.dot(np.roll(x, j, axis=0), con[j])

    # polynomial layer : w_1 * u + w_2 * |u|^2 * u  + w_3 * |u|^4 * u
    y = np.zeros((n_samples, n), dtype=np.complex128)

    for j in range(p):
        y += w[j] * np.power(np.abs(u), 2*j+1) * u * (poly_power ** j)

    # Add noise
    noise = noise_level * np.random.randn(n_samples, n) + 1j*noise_level * np.random.randn(n_samples, n)
    y += noise
    
    return x, y

# Example usage
n_samples = 300
# a series of matrix: k x m x n 
# k : influence scope 5
# m : output dimension  3
# n : input dimension 2
# p : polynomial degree 1, 3, 5, 3 coefficients in total
k = 2
m = 3
n = 2
p = 3

true_coefficients = [0,1 ]
### part 1 : conv layer
true_coefficients[0] = [np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]
### part 2 : polynomial layer
true_coefficients[1] = np.random.randn(p, n) + 1j*np.random.randn(p, n)
noise_level = 0.05
poly_power = 0.05
x, y = generate_data(n_samples, true_coefficients, noise_level,poly_power)

print("Input signal:", x[:3])
print("Desired signal:", y[:3])

Input signal: [[ 0.19766009-1.36181592j  1.33484857+0.16652728j -0.08687561+1.16975154j]
 [ 1.56153229+0.5594802j  -0.30585302+1.12469699j -0.47773142-0.92626446j]
 [ 0.10073819+1.14597196j  0.35543847-0.70112464j  0.26961241+1.13017134j]]
Desired signal: [[ 54.04390924-21.26275417j  -1.02177033 -4.0947404j ]
 [-27.37812204 -4.30378057j  -4.0005473  -3.28833761j]
 [ 52.68748463-42.75433476j   2.24270581 -0.83834891j]]


In [66]:
### LMS filter
## Hyperparameters
k = len(true_coefficients[0]) ## influence scope
p = len(true_coefficients[1]) ## polynomial degree
mu = 0.01


def lms_filter(x, y, k, mu, p):
    Ws = [] 
    m, n = x.shape[1], y.shape[1]

    ConvW = np.array([np.random.randn(m, n) + 1j*np.random.randn(m, n) for _ in range(k)]) ## k x m x n
    PolyW = np.random.randn(p, n) + 1j*np.random.randn(p, n) ## p x n
    poly_power_limit = 0.01 
    # e = np.zeros((len(x), n), dtype=complex) ## record error

    for i in range(k, len(x)):
        y_hat = np.zeros((n), dtype=complex)
        u = np.zeros((n), dtype=complex)
        # convolution layer
        for j in range(k):
            u += np.dot(x[i-j], ConvW[j])

        # polynomial layer
        X_Poly = [np.power(np.abs(u), 2*j+1) * u* (poly_power_limit ** j) for j in range(p)] 
        print(X_Poly)
        for j in range(p):
            y_hat += PolyW[j] * X_Poly[j] 
        
        error = y[i] - y_hat 
        # e[i] = error
        # update PolyW, eg:1, 3, 5 
        for j in range(p):
            PolyW[j] += mu * np.dot(np.conj(X_Poly[j]), error)
        # update ConvW
        for j in range(k):
            ConvW[j] += mu * np.outer(np.conj(x[i-j]) , error)
        Ws.append((ConvW.copy(), PolyW.copy()))
    return  Ws


In [67]:
### Example usage
Ws = lms_filter(x, y, k, mu, p)

## print in 3 decimal places
np.set_printoptions(precision=3)

print("Estimated coefficients:", Ws[0][0])
print("True coefficients:", true_coefficients[0])

print("Estimated coefficients:", Ws[0][1])
print("True coefficients:", true_coefficients[1])

[array([-8.931+63.027j,  3.03  -4.908j]), array([-5.685+40.121j,  0.175 -0.283j]), array([-3.619+2.554e+01j,  0.01 -1.633e-02j])]
[array([-19.42 +37.057j, -13.184+13.11j ]), array([-8.124+15.503j, -2.451 +2.438j]), array([-3.399+6.486j, -0.456+0.453j])]
[array([22285.565+20123.439j,  2001.375 +3935.494j]), array([6691606.751+6042392.886j,   88363.835 +173758.228j]), array([2.009e+09+1.814e+09j, 3.901e+06+7.672e+06j])]
[array([-9.322e+20+9.049e+20j, -1.343e+16+5.620e+15j]), array([-1.211e+40+1.176e+40j, -1.956e+30+8.184e+29j]), array([-1.574e+59+1.527e+59j, -2.848e+44+1.192e+44j])]
[array([8.719e+153+1.254e+154j, 6.998e+123+2.945e+124j]), array([       nan       +infj, 2.118e+246+8.915e+246j]), array([nan+nanj, nan+nanj])]
[array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj])]
[array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj])]
[array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj]), array([nan+nanj, nan+nanj])]
[arr

C:\Users\mioshi\AppData\Local\Temp\ipykernel_40564\3451268448.py:26: RuntimeWarning: overflow encountered in multiply
  X_Poly = [np.power(np.abs(u), 2*j+1) * u* (poly_power_limit ** j) for j in range(p)]
C:\Users\mioshi\AppData\Local\Temp\ipykernel_40564\3451268448.py:26: RuntimeWarning: invalid value encountered in multiply
  X_Poly = [np.power(np.abs(u), 2*j+1) * u* (poly_power_limit ** j) for j in range(p)]
C:\Users\mioshi\AppData\Local\Temp\ipykernel_40564\3451268448.py:26: RuntimeWarning: overflow encountered in power
  X_Poly = [np.power(np.abs(u), 2*j+1) * u* (poly_power_limit ** j) for j in range(p)]
C:\Users\mioshi\AppData\Local\Temp\ipykernel_40564\3451268448.py:29: RuntimeWarning: overflow encountered in multiply
  y_hat += PolyW[j] * X_Poly[j]


In [55]:
print("Estimated coefficients:", Ws[-1][0])
print("True coefficients:", true_coefficients[0])

print("Estimated coefficients:", Ws[-1][1])
print("True coefficients:", true_coefficients[1])

Estimated coefficients: [[[nan+nanj nan+nanj]
  [nan+nanj nan+nanj]
  [nan+nanj nan+nanj]]

 [[nan+nanj nan+nanj]
  [nan+nanj nan+nanj]
  [nan+nanj nan+nanj]]]
True coefficients: [array([[ 0.441-0.909j, -0.331-0.592j],
       [ 2.431+0.188j, -0.252-0.33j ],
       [ 0.11 -1.193j,  1.582-0.205j]]), array([[-0.359-1.511j,  0.603+0.645j],
       [-1.665-0.981j, -0.7  -0.857j],
       [ 1.151-0.872j,  1.857-0.423j]])]
Estimated coefficients: [[nan+nanj nan+nanj]
 [nan+nanj nan+nanj]
 [nan+nanj nan+nanj]]
True coefficients: [[ 0.996+0.793j  0.712-0.632j]
 [ 0.059-0.006j -0.363-0.101j]
 [ 0.003-0.052j -0.106+0.249j]]
